In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
import time
from tqdm import tqdm
from IPython.display import Image
import regex as re
from IPython.core.display import display,HTML
from joblib import Parallel, delayed
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
PATH=ChromeDriverManager().install()

In [3]:
# EL REFUGIO

In [4]:
url = 'https://elrefugio.org/adopta/perros'
driver=webdriver.Chrome(PATH) 
driver.get(url)
time.sleep(3)

muchos=driver.find_elements(By.CLASS_NAME, 'item')
uno = [e.find_element(By.TAG_NAME, 'a') for e in muchos]
links_elrefugio = [e.get_attribute('href') for e in uno]

imagen=[]
nombres= []
carac= []
protec_id=[]
for e in tqdm(links_elrefugio):
    protec_id.append(1)
    driver.get(e)
    box = driver.find_element(By.CLASS_NAME, 'boxSlideFotos')
    img = box.find_element(By.TAG_NAME, 'img')
    imagen.append(img.get_attribute('src'))
    desc = driver.find_element(By.CLASS_NAME, 'txtDoble')
    nombres.append(desc.find_element(By.TAG_NAME, 'h1').text)
    todo = desc.find_elements(By.TAG_NAME, 'li')
    for i in todo:
        i = i.text
        i = i.split(':')
        i.pop(0)
        carac += i
        
driver.quit()

n = 6
caracter = [carac[i:i + n] for i in range(0, len(carac), n)]

100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [00:49<00:00,  1.44it/s]


In [5]:
colnames= ['SEXO', 'EDAD', 'NACIMIENTO', 'RAZA', 'TAMAÑO', 'CARACTER']
refugio = pd.DataFrame(caracter, columns=colnames)

refugio['NOMBRE']= nombres
refugio['PROTECTORA']= protec_id
refugio['LINK']= links_elrefugio

sexo= []
for e in refugio.SEXO:
    e = e.lower()
    e= e.strip()
    sexo.append(e)
refugio['SEXO']= sexo

raza= []
for e in refugio.RAZA:
    e = e.lower()
    e= e.strip()
    raza.append(e)
refugio['RAZA']= raza

tam= []
for e in refugio.TAMAÑO:
    e = e.lower()
    e= e.strip()
    tam.append(e)
refugio['TAMAÑO']= tam

car= []
for e in refugio.CARACTER:
    e = e.lower()
    e= e.strip()
    car.append(e)
refugio['CARACTER']= car

nom= []
for e in refugio.NOMBRE:
    e = e.capitalize()
    e= e.strip()
    nom.append(e)
refugio['NOMBRE']= nom

nac= []
for e in refugio.NACIMIENTO:
    e= e.strip()
    nac.append(e)
refugio['NACIMIENTO']= nac




images1 = imagen
refugio['FOTO'] = images1

# convert your links to html tags 
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

pd.set_option('display.max_colwidth', None)

image_cols = ['FOTO']  #<- define which columns will be used to convert to html

# Create the dictionariy to be passed as formatters
format_dict = {}
for e in image_cols:
    format_dict[e] = path_to_image_html


#display(HTML(perros.to_html(escape=False ,formatters=format_dict)))

refugio.RAZA.replace({'galga': 'galgo', 'mestiza': 'mestizo'}, inplace= True)
refugio.TAMAÑO.replace({'medio': 'mediano', 'mediana': 'mediano'}, inplace= True)
refugio.CARACTER.replace({'muy cariñosa': 'cariñosa', 'timida': 'tímida'}, inplace= True)
refugio['NACIMIENTO'] = pd.to_datetime(refugio['NACIMIENTO'])
refugio['NACIMIENTO'] = (refugio['NACIMIENTO']).dt.year
refugio['HOY'] = pd.datetime.now().date()
refugio['HOY'] = pd.to_datetime(refugio['HOY'])
refugio['HOY'] = (refugio['HOY']).dt.year
refugio['AÑOS'] = (refugio['HOY'] - refugio['NACIMIENTO'])
refugio.drop(['NACIMIENTO', 'HOY', 'EDAD'], axis = 1, inplace = True)
